In [10]:
# Removes large-scale background gradients from the input flc files

import glob
import os

from astropy.io import fits
from astropy.stats import SigmaClip, sigma_clip
import numpy as np
from photutils import Background2D, MedianBackground

# The files to remove the background gradient from
files = glob.glob('*flc.fits')

# Subtract the background gradient from each of the above files
for f in files:
    print('Working on {} ...'.format(os.path.basename(f)))
    h = fits.open(f)
    for ext in [1,4]:
        print('\tWorking on extension {}...'.format(ext))
        data_orig = np.copy(h[ext].data)
        data = h[ext].data

        # Subtract off median
        clipped = sigma_clip(data, sigma=3, iters=5)
        data = data - np.nanmedian(clipped)

        # Find the background gradient
        print('\tFinding the background gradient...')
        s = SigmaClip(sigma=3.)
        bkg_estimator = MedianBackground()
        bkg = Background2D(data, (150, 150), filter_size=(10, 10), 
                           sigma_clip=s, bkg_estimator=bkg_estimator)
        skydark = bkg.background
        fits.writeto('{}_bkg_ext{}.fits'.format(os.path.basename(f).split('_')[0], ext), 
                     skydark, overwrite=True)

        # Subtract the background gradient from the image
        print('\tSubtracting the background gradient')
        data_new = data_orig - skydark
        h[ext].data = data_new.astype('float32')

    h.writeto(f, overwrite=True)
    print('Finished removing background gradient from {}'.format(os.path.basename(f)))


Working on jdxfa9inq_flc.fits ...
	Working on extension 1...


/Users/bsunnquist/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:639: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


	Finding the background gradient...
	Subtracting the background gradient
	Working on extension 4...
	Finding the background gradient...
	Subtracting the background gradient
Finished removing background gradient from jdxfa9inq_flc.fits
Working on jdxfa9hxq_flc.fits ...
	Working on extension 1...
	Finding the background gradient...
	Subtracting the background gradient
	Working on extension 4...
	Finding the background gradient...
	Subtracting the background gradient
Finished removing background gradient from jdxfa9hxq_flc.fits
Working on jdxfa9ikq_flc.fits ...
	Working on extension 1...
	Finding the background gradient...
	Subtracting the background gradient
	Working on extension 4...
	Finding the background gradient...
	Subtracting the background gradient
Finished removing background gradient from jdxfa9ikq_flc.fits
